<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Start-to-Finish Example: `GiRaFFE_NRPy` 1D tests

### Authors: Patrick Nelson & Terrence Pierre Jacques

### Adapted from [Start-to-Finish Example: Head-On Black Hole Collision](../Tutorial-Start_to_Finish-BSSNCurvilinear-Two_BHs_Collide.ipynb)

## This module compiles and runs code tests for all 1D initial data options available in GiRaFFE-NRPy+, evolving one-dimensional GRFFE waves.

### NRPy+ Source Code for this module: 

* Main python module for all 1D initial data: [GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_1D_tests.py](../../edit/in_progress/GiRaFFEfood_NRPy/GiRaFFEfood_NRPy_1D_tests.py) __Options:__
    1. [Fast Wave](Tutorial-GiRaFFEfood_NRPy_1D_tests-fast_wave.ipynb)
    1. [Alfven Wave](Tutorial-GiRaFFEfood_NRPy_1D_alfven_wave.ipynb)
    1. [Degenerate Alfven Wave](Tutorial-GiRaFFEfood_NRPy_1D_tests-degen_Alfven_wave.ipynb)
    1. [Three Alfven Waves](Tutorial-GiRaFFEfood_NRPy_1D_tests-three_waves.ipynb)
    1. [FFE Breakdown](Tutorial-GiRaFFEfood_NRPy_1D_tests-FFE_breakdown.ipynb)
* [GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_Afield_flux.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_Afield_flux.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy_staggered-Afield_flux.ipynb) Generates the expressions to find the flux term of the induction equation.
* [GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_A2B.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_A2B.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy_staggered-A2B.ipynb) Generates the driver to compute the magnetic field from the vector potential/
* [GiRaFFE_NRPy/GiRaFFE_NRPy_BCs.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_BCs.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy-BCs.ipynb) Generates the code to apply boundary conditions to the vector potential, scalar potential, and three-velocity.
* [GiRaFFE_NRPy/GiRaFFE_NRPy_C2P_P2C.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_C2P_P2C.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy-C2P_P2C.ipynb) Generates the conservative-to-primitive and primitive-to-conservative solvers.
* [GiRaFFE_NRPy/GiRaFFE_NRPy_Metric_Face_Values.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_Metric_Face_Values.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy-Metric_Face_Values.ipynb) Generates code to interpolate metric gridfunctions to cell faces.
* [GiRaFFE_NRPy/GiRaFFE_NRPy_PPM.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_PPM.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy-PPM.ipynb) Genearates code to reconstruct primitive variables on cell faces.
* [GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_Source_Terms.py](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_staggered_Source_Terms.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy_staggered-Source_Terms.ipynb) Generates the expressions to find the flux term of the Poynting flux evolution equation.
* [GiRaFFE_NRPy/Stilde_flux.py](../../edit/in_progress/GiRaFFE_NRPy/Stilde_flux.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy-Stilde_flux.ipynb) Generates the expressions to find the flux term of the Poynting flux evolution equation.
* [../GRFFE/equations.py](../../edit/GRFFE/equations.py) [\[**tutorial**\]](../Tutorial-GRFFE_Equations-Cartesian.ipynb) Generates code necessary to compute the source terms.
* [../GRHD/equations.py](../../edit/GRHD/equations.py) [\[**tutorial**\]](../Tutorial-GRHD_Equations-Cartesian.ipynb) Generates code necessary to compute the source terms.

Here we use NRPy+ to generate the C source code necessary to set up initial data for an Alfv&eacute;n wave (see [the original GiRaFFE paper](https://arxiv.org/pdf/1704.00599.pdf)). Then we use it to generate the RHS expressions for [Method of Lines](https://reference.wolfram.com/language/tutorial/NDSolveMethodOfLines.html) time integration based on the [explicit Runge-Kutta fourth-order scheme](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) (RK4).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Set core NRPy+ parameters for numerical grids
1. [Step 2](#grffe): Output C code for GRFFE evolution
    1. [Step 2.a](#mol): Output macros for Method of Lines timestepping
1. [Step 3](#gf_id): Import `GiRaFFEfood_NRPy` initial data modules
1. [Step 4](#cparams): Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h`
1. [Step 5](#mainc): `GiRaFFE_NRPy_standalone.c`: The Main C Code
1. [Step 6](#compileexec): Compile and execute C codes
1. [Step 7](#plots): Data Visualization
1. [Step 8](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='setup'></a>

# Step 1: Set up core functions and parameters for solving  GRFFE equations \[Back to [top](#toc)\]
$$\label{setup}$$


In [1]:
import shutil, os, sys           # Standard Python modules for multiplatform OS-level functions
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
GRMHD_dir_path = os.path.join("../GRMHD_formulation/")
sys.path.append(GRMHD_dir_path)
import GRMHD_equations_new_version as GRMHD    # NRPy+: Generate general relativistic magnetohydrodynamics equations

src_dir_path = os.path.join("../source_terms/")
sys.path.append(src_dir_path)
import IGM_All_Source_Terms as ST   

fluxes_dir_path = os.path.join("../fluxes/")
sys.path.append(fluxes_dir_path)
import IGM_All_fluxes as FL   
import IGM_Characteristic_Speeds as chsp 


nrpy_dir_path = os.path.join("../nrpy/nrpytutorial/")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
# Step P1: Import needed NRPy+ core modules:
from outputC import nrpyAbs, lhrh, outputC, outCfunction, add_to_Cfunction_dict, lhrh # NRPy+: Core C code output module
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import loop as lp                # NRPy+: Generate C code loops
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

import CurviBoundaryConditions.CurviBoundaryConditions_new_way as CBC

# Step P2: Create C code output directory:
Ccodesrootdir = os.path.join("IGM_Flux_SRC_UnitTest_standalone_Ccodes_fixes_flux/")
# First remove C code output directory if it exists
# Courtesy https://stackoverflow.com/questions/303200/how-do-i-remove-delete-a-folder-that-is-not-empty
# !rm -r ScalarWaveCurvilinear_Playground_Ccodes
shutil.rmtree(Ccodesrootdir, ignore_errors=True)
# # Then create a fresh directory
cmd.mkdir(Ccodesrootdir)

# Step P3: Create executable output directory:
outdir = os.path.join(Ccodesrootdir, "output")
cmd.mkdir(outdir)

# Step P4: Enable/disable SIMD. If enabled, code should run ~2x faster on most CPUs.
enable_SIMD = False

# Step P5: Enable reference metric precomputation.
enable_rfm_precompute = False
par.set_parval_from_str("reference_metric::rfm_precompute_to_Cfunctions_and_NRPy_basic_defines", "True")

if enable_SIMD and not enable_rfm_precompute:
    print("ERROR: SIMD does not currently handle transcendental functions,\n")
    print("       like those found in rfmstruct (rfm_precompute).\n")
    print("       Therefore, enable_SIMD==True and enable_rfm_precompute==False\n")
    print("       is not supported.\n")
    sys.exit(1)

# Step P6: Enable "FD functions". In other words, all finite-difference stencils
#         will be output as inlined static functions. This is essential for
#         compiling highly complex FD kernels with using certain versions of GCC;
#         GCC 10-ish will choke on BSSN FD kernels at high FD order, sometimes
#         taking *hours* to compile. Unaffected GCC versions compile these kernels
#         in seconds. FD functions do not slow the code performance, but do add
#         another header file to the C source tree.
# With gcc 7.5.0, enable_FD_functions=True decreases performance by 10%
enable_FD_functions = False

# Step 1: Set some core parameters, including CoordSystem, boundary condition,
#                                             MoL, timestepping algorithm, FD order,
#                                             floating point precision, and CFL factor:

# Step 1.a: Set the coordinate system for the numerical grid
# Choices are: Spherical, SinhSpherical, SinhSphericalv2, Cylindrical, SinhCylindrical,
#              SymTP, SinhSymTP
CoordSystem     = "Cartesian"
par.set_parval_from_str("reference_metric::CoordSystem", CoordSystem)
rfm.reference_metric()

# Step 1.b: Set outer boundary condition
# Current choices are extrapolation (quadratic polynomial extrapolation) or radiation
# OuterBoundaryCondition = "radiation"
OuterBoundaryCondition = "extrapolation"

# Step 1.c: Set defaults for Coordinate system parameters.
#           These are perhaps the most commonly adjusted parameters,
#           so we enable modifications at this high level.

# domain_size sets the default value for:
#   * Spherical's params.RMAX
#   * SinhSpherical*'s params.AMAX
#   * Cartesians*'s -params.{x,y,z}min & .{x,y,z}max
#   * Cylindrical's -params.ZMIN & .{Z,RHO}MAX
#   * SinhCylindrical's params.AMPL{RHO,Z}
#   * *SymTP's params.AMAX
domain_size     = 10.0 # Needed for all coordinate systems.

# sinh_width sets the default value for:
#   * SinhSpherical's params.SINHW
#   * SinhCylindrical's params.SINHW{RHO,Z}
#   * SinhSymTP's params.SINHWAA
sinh_width      = 0.4 # If Sinh* coordinates chosen

# sinhv2_const_dr sets the default value for:
#   * SinhSphericalv2's params.const_dr
#   * SinhCylindricalv2's params.const_d{rho,z}
sinhv2_const_dr = 0.05# If Sinh*v2 coordinates chosen

# SymTP_bScale sets the default value for:
#   * SinhSymTP's params.bScale
SymTP_bScale    = 1.0 # If SymTP chosen

# Step 1.d: Set the order of spatial and temporal derivatives;
#           the core data type, and the CFL factor.
# RK_method choices include: Euler, "RK2 Heun", "RK2 MP", "RK2 Ralston", RK3, "RK3 Heun", "RK3 Ralston",
#              SSPRK3, RK4, DP5, DP5alt, CK5, DP6, L6, DP8
RK_method = "RK4"
FD_order  = 6        # Finite difference order: even numbers only, starting with 2. 12 is generally unstable
REAL      = "double" # Best to use double here.
CFL_FACTOR= 0.5


# Step 5: Set the finite differencing order to FD_order (set above).
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", FD_order)

thismodule = __name__
TINYDOUBLE = par.Cparameters("REAL", thismodule, "TINYDOUBLE", 1e-100)


<a id='grffe'></a>

# Step 2: Output C code for GRFFE evolution \[Back to [top](#toc)\]
$$\label{grffe}$$

We will first write the C codes needed for GRFFE evolution. We have already written a module to generate all these codes and call the functions in the appropriate order, so we will import that here. We will take the slightly unusual step of doing this before we generate the initial data functions because the main driver module will register all the gridfunctions we need. It will also generate functions that, in addition to their normal spot in the MoL timestepping, will need to be called during the initial data step to make sure all the variables are appropriately filled in. 

All of this is handled with a single call to `GiRaFFE_NRPy_Main_Driver_generate_all()`, which will register gridfunctions, write all the C code kernels, and write the C code functions to call those.

In [2]:
alpha = gri.register_gridfunctions("AUXEVOL","alpha")
gammaDD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","gammaDD","sym01",DIM=3)
betaU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","betaU", DIM=3)

alpha_dD = ixp.declarerank1("alpha_dD", DIM=3)
gammaDD_dD = ixp.declarerank3("gammaDD_dD","sym01",DIM=3)
betaU_dD = ixp.declarerank2("betaU_dD", "nosym", DIM=3)

tmp1D = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","tmp1D",DIM=3)
tmp2DD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","tmp2DD","nosym", DIM=3)
tmp3DDD = ixp.register_gridfunctions_for_single_rank3("AUXEVOL","tmp3DDD","sym01", DIM=3)

expr_list = []
for i in range(3):
    expr_list.append(lhrh(lhs=gri.gfaccess("out_gfs",fr"tmp1D{i}"),rhs=alpha_dD[i]))
    for j in range(3):
        expr_list.append(lhrh(lhs=gri.gfaccess("out_gfs",fr"tmp2DD{i}{j}"),rhs=betaU_dD[i][j]))

for j in range(3):
    i = 0
    for k in range(3):
        expr_list.append(lhrh(lhs=gri.gfaccess("out_gfs",fr"tmp3DDD{i}{j}{k}"),rhs=gammaDD_dD[i][j][k]))       

for j in range(1,3):
    i = 1
    for k in range(3):
        expr_list.append(lhrh(lhs=gri.gfaccess("out_gfs",fr"tmp3DDD{i}{j}{k}"),rhs=gammaDD_dD[i][j][k]))      

i = 2
j = 2
for k in range(3):
    expr_list.append(lhrh(lhs=gri.gfaccess("out_gfs",fr"tmp3DDD{i}{j}{k}"),rhs=gammaDD_dD[i][j][k]))

outCparams = "outCverbose=False,CSE_sorting=canonical,CSE_enable=True"
desc = "Calculate metric derivatives"
includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
name = "calculate_metric_derivatives"
body = fin.FD_outputC("returnstring",
                      expr_list,
                      params=outCparams)
body = body.replace("TMP1D", "ALPHA_DD")
body = body.replace("TMP2DD", "BETAU_DD")
body = body.replace("TMP3DDD", "GAMMADD_DD")

c_type = "void"
params   ="const paramstruct *restrict params, "
params += "REAL *restrict xx[3], "
params += "REAL *restrict auxevol_gfs"
loopopts = "InteriorPoints"
loopopts += ",Read_xxs"
add_to_Cfunction_dict(
    includes=includes,
    desc=desc,
    c_type=c_type, name=name, params=params,
    body=body,
    rel_path_to_Cparams=os.path.join("."), loopopts = loopopts)

gri.glb_gridfcs_list = []

In [3]:
formalism="ADM"

alpha_face = gri.register_gridfunctions("AUXEVOL","alpha_face")
# cf_face = gri.register_gridfunctions("AUXEVOL","cf_face")
gamma_faceDD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","gamma_faceDD","sym01",DIM=3)
beta_faceU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","beta_faceU", DIM=3)

alpha = gri.register_gridfunctions("AUXEVOL","alpha")
alpha_dD = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","alpha_dD", DIM=3)

# cf = gri.register_gridfunctions("AUXEVOL","cf")

gammaDD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","gammaDD","sym01",DIM=3)
gammaDD_dD = ixp.register_gridfunctions_for_single_rank3("AUXEVOL","gammaDD_dD","sym01",DIM=3)

KDD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","KDD","sym01",DIM=3)
# KDD_dD = ixp.register_gridfunctions_for_single_rank3("AUXEVOL","KDD_dD","sym01",DIM=3)

betaU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","betaU", DIM=3)
betaU_dD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","betaU_dD", "nosym", DIM=3)

# We'll need some more gridfunctions, now, to represent the reconstructions of BU and ValenciavU
# on the right and left faces
u4_rU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","u4_rU",DIM=4)
V_rU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","V_rU",DIM=3)
B_rU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","B_rU",DIM=3)

u4_lU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","u4_lU",DIM=4)
V_lU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","V_lU",DIM=3)
B_lU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","B_lU",DIM=3)

u4U = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","u4U",DIM=4)
VU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","VU",DIM=3)
BU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","BU",DIM=3)

rho_star = gri.register_gridfunctions("EVOL","rho_star")
tau_tilde = gri.register_gridfunctions("EVOL","tau_tilde")
StildeD = ixp.register_gridfunctions_for_single_rank1("EVOL","StildeD")

HLLE_flux_StildeD = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","HLLE_flux_StildeD")
HLLE_flux_rho_star = gri.register_gridfunctions("AUXEVOL","HLLE_flux_rho_star")
HLLE_flux_tau_tilde = gri.register_gridfunctions("AUXEVOL","HLLE_flux_tau_tilde")

h = gri.register_gridfunctions("AUXEVOL", "h")
h_r = gri.register_gridfunctions("AUXEVOL", "h_r")
h_l = gri.register_gridfunctions("AUXEVOL", "h_l")

P = gri.register_gridfunctions("AUXEVOL", "P")
P_r = gri.register_gridfunctions("AUXEVOL", "P_r")
P_l = gri.register_gridfunctions("AUXEVOL", "P_l")

rho_b = gri.register_gridfunctions("AUXEVOL", "rhob")
rho_b_r = gri.register_gridfunctions("AUXEVOL", "rhob_r")
rho_b_l = gri.register_gridfunctions("AUXEVOL", "rhob_l")

Gamma_th_r = gri.register_gridfunctions("AUXEVOL", "Gamma_th_r")
Gamma_th_l = gri.register_gridfunctions("AUXEVOL", "Gamma_th_l")

epsilon_th_r = gri.register_gridfunctions("AUXEVOL", "epsilon_th_r")
epsilon_th_l = gri.register_gridfunctions("AUXEVOL", "epsilon_th_l")

dPcold_drhob_r = gri.register_gridfunctions("AUXEVOL", "dPcold_drhob_r")
dPcold_drhob_l = gri.register_gridfunctions("AUXEVOL", "dPcold_drhob_l")


In [4]:
ST.add_to_Cfunction_dict__Stilde_SourceTerms(formalism=formalism)
chsp.add_to_Cfunction_dict__GRMHD_characteristic_speeds(formalism=formalism)
FL.add_to_Cfunction_dict__GRMHD_fluxes(formalism=formalism)

<a id='mol'></a>

## Step 2.a: Output macros for Method of Lines timestepping \[Back to [top](#toc)\]
$$\label{mol}$$

Now, we generate the code to implement the method of lines using the fourth-order Runge-Kutta algorithm.

<a id='cparams'></a>

# Step 4: Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h` \[Back to [top](#toc)\]
$$\label{cparams}$$

Based on declared NRPy+ Cparameters, first we generate `declare_Cparameters_struct.h`, `set_Cparameters_default.h`, and `set_Cparameters[-SIMD].h`.

Then we output `free_parameters.h`, which sets initial data parameters, as well as grid domain & reference metric parameters, applying `domain_size` and `sinh_width`/`SymTP_bScale` (if applicable) as set above

In [5]:
# Step 1.c.i: Set free_parameters.h
outstr = r"""
// Free parameters related to physical system:
"""

# Append to $Ccodesrootdir/free_parameters.h reference metric parameters based on generic
#    domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale,
#    parameters set above.
outstr += rfm.out_default_free_parameters_for_rfm("returnstring",
                                                  domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale)
with open(os.path.join(Ccodesrootdir,"free_parameters.h"),"w") as file:
    file.write(outstr.replace("params.", "griddata.params."))

In [6]:
GRMHD.set_up_base_vars(formalism=formalism)

GRMHD.compute_vU_from_u4U__no_speed_limit(GRMHD.u4U)   

GRMHD.compute_sqrtgammaDET(GRMHD.gammaDD)
GRMHD.compute_smallb4U(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.u4U, GRMHD.BU, GRMHD.sqrt4pi)
GRMHD.compute_smallbsquared(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.smallb4U)

# First compute stress-energy tensor T4UU and T4UD:
GRMHD.compute_T4UU(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.rho_b,GRMHD.P,GRMHD.h,GRMHD.u4U, GRMHD.smallb4U, GRMHD.smallbsquared)
GRMHD.compute_T4UD(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.T4UU)

# Compute conservative variables in terms of primitive variables
GRMHD.compute_rho_star(GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.rho_b,GRMHD.u4U)
GRMHD.compute_tau_tilde(GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.T4UU,GRMHD.rho_star)
GRMHD.compute_S_tildeD(GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.T4UD)

# Next compute fluxes of conservative variables
GRMHD.compute_rho_star_fluxU(GRMHD.VU,GRMHD.rho_star)
GRMHD.compute_tau_tilde_fluxU(GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.VU, GRMHD.T4UU, GRMHD.rho_star)
GRMHD.compute_S_tilde_fluxUD (GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.T4UD)

# Then declare derivatives & compute g4DD_zerotimederiv_dD
GRMHD.compute_g4DD_zerotimederiv_dD(GRMHD.gammaDD,GRMHD.betaU,GRMHD.alpha, GRMHD.gammaDD_dD,GRMHD.betaU_dD,GRMHD.alpha_dD)

# Then compute source terms on tau_tilde and S_tilde equations
GRMHD.compute_tau_tilde_source_term(GRMHD.KDD,GRMHD.betaU,GRMHD.alpha, GRMHD.sqrtgammaDET, GRMHD.alpha_dD, GRMHD.T4UU)
GRMHD.compute_S_tilde_source_termD(GRMHD.alpha,GRMHD.sqrtgammaDET,GRMHD.g4DD_zerotimederiv_dD, GRMHD.T4UU)


tau_tilde_source_term_free_symbols = GRMHD.tau_tilde_source_term.free_symbols
S_tilde_source_termD0_free_symbols = GRMHD.S_tilde_source_termD[0].free_symbols
S_tilde_source_termD1_free_symbols = GRMHD.S_tilde_source_termD[1].free_symbols
S_tilde_source_termD2_free_symbols = GRMHD.S_tilde_source_termD[2].free_symbols

all_free_sysmbols = tau_tilde_source_term_free_symbols.union(\
                             S_tilde_source_termD0_free_symbols, 
                             S_tilde_source_termD1_free_symbols,
                             S_tilde_source_termD2_free_symbols)       

In [7]:
local_prims_vectors = ["u4U0", "u4U1", "u4U2", "u4U3",
               "BU0", "BU1", "BU2"]

local_prims_scalars = ["P", "h", "rhob", "Gamma_th", "epsilon_th", "dPcold_drhob"]

local_metric_vars = ["alpha", "betaU0", "betaU1", "betaU2",
                    "gammaDD01", "gammaDD02", "gammaDD12", 
                    "gammaDD00", "gammaDD11", "gammaDD22",
                    "KDD01", "KDD02", "KDD12", 
                    "KDD00", "KDD11", "KDD22"]

loop_body = r"""
"""

for p_v in local_prims_vectors:
    #capitalize str
    P_V = p_v.upper()
    loop_body += fr"""    
    reconstructed_prims->{p_v} = auxevol_gfs[IDX4ptS({P_V}GF, idx)];
    reconstructed_prims_r->{p_v} = auxevol_gfs[IDX4ptS({p_v.replace("U", "_RU").upper()}GF, idx)];
    reconstructed_prims_l->{p_v} = auxevol_gfs[IDX4ptS({p_v.replace("U", "_LU").upper()}GF, idx)];
"""
    
for p_v in local_prims_scalars:
    #capitalize str
    P_V = p_v.upper()
    loop_body += fr"""
    reconstructed_prims_r->{p_v} = auxevol_gfs[IDX4ptS({P_V}_RGF, idx)];
    reconstructed_prims_l->{p_v} = auxevol_gfs[IDX4ptS({P_V}_LGF, idx)];
"""
    if p_v!="Gamma_th" and p_v!="epsilon_th" and p_v!="dPcold_drhob":
        loop_body += fr"""
    reconstructed_prims->{p_v} = auxevol_gfs[IDX4ptS({P_V}GF, idx)];
"""
        
for m_v in local_metric_vars:
    m_v_face = m_v.replace("alpha", "alpha_face").replace("U", "_faceU").replace("DD", "_faceDD")
    M_V = m_v.upper()
    M_V_face = m_v_face.upper()
    loop_body += fr"""    
    metric_quantities->{m_v} = auxevol_gfs[IDX4ptS({M_V}GF, idx)];
"""
    if "KDD" not in m_v:
        loop_body += fr""" 
    metric_face_quantities->{m_v_face} = auxevol_gfs[IDX4ptS({M_V_face}GF, idx)];
"""
    
for var in all_free_sysmbols:
    if "_dD" in str(var):
        loop_body += fr"""
    metric_quantities_derivatives->{str(var)} = auxevol_gfs[IDX4ptS({str(var).upper()}GF, idx)];
"""

c_type = "void"
params   = "const int idx, const paramstruct *params, const double *restrict auxevol_gfs, "
params  += "reconstructed_prims_struct *restrict reconstructed_prims, "
params  += "reconstructed_prims_struct *restrict reconstructed_prims_r, "
params  += "reconstructed_prims_struct *restrict reconstructed_prims_l, " 
params  += "metric_quantities_struct *restrict metric_quantities, "
params  += "metric_face_quantities_struct *restrict metric_face_quantities, "
params  += "metric_quantities_derivatives_struct *restrict metric_quantities_derivatives"

desc = "Initialize structs used for flux and source calculations"
name = "initialize_structs"
includes = ["NRPy_basic_defines.h"]

add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        name=name,
        params=params,
        body=loop_body, 
        rel_path_to_Cparams=os.path.join("."))

In [8]:
c_type = "void"
params   = "const char *restrict binary_file, const paramstruct *params, double *restrict auxevol_gfs"
desc = "Read in binary data for metric quantities and primitives"
name = "read_from_binary_file_all"
includes = ["NRPy_basic_defines.h"]

body = r"""
  const int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;

  FILE *infile = fopen(binary_file, "rb");

  double correct_magic_number = 1.130814081305130e-9;
  double magic_number1, magic_number2, magic_number3, magic_number4, magic_number5;
  
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*RHOBGF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*PGF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*RHOB_RGF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*P_RGF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*RHOB_LGF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*P_LGF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(&magic_number1, sizeof(double), 1, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*VU0GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*VU1GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*VU2GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*V_RU0GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*V_RU1GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*V_RU2GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*V_LU0GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*V_LU1GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*V_LU2GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(&magic_number2, sizeof(double), 1, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*BU0GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*BU1GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*BU2GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*B_RU0GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*B_RU1GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*B_RU2GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*B_LU0GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*B_LU1GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*B_LU2GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(&magic_number3, sizeof(double), 1, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*BETAU0GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*BETAU1GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*BETAU2GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*GAMMADD00GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*GAMMADD01GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*GAMMADD02GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*GAMMADD11GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*GAMMADD12GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*GAMMADD22GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*ALPHAGF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(&magic_number4, sizeof(double), 1, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*KDD00GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*KDD01GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*KDD02GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*KDD11GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*KDD12GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*KDD22GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(&magic_number5, sizeof(double), 1, infile);
  fclose(infile);
  if(magic_number1!=correct_magic_number){ printf("ERROR: magic_number1 does not match"); exit(1);}
  if(magic_number2!=correct_magic_number){ printf("ERROR: magic_number2 does not match"); exit(1);}
  if(magic_number3!=correct_magic_number){ printf("ERROR: magic_number3 does not match"); exit(1);}
  if(magic_number4!=correct_magic_number){ printf("ERROR: magic_number4 does not match"); exit(1);}
  if(magic_number5!=correct_magic_number){ printf("ERROR: magic_number5 does not match"); exit(1);}
"""

add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        name=name,
        params=params,
        body=body, 
        rel_path_to_Cparams=os.path.join("."))

In [9]:
c_type = "void"
params   = "const char *restrict binary_file, const paramstruct *params, double *restrict auxevol_gfs"
desc = "Read in binary data for primitives"
name = "read_from_binary_file_recons"
includes = ["NRPy_basic_defines.h"]

body = r"""
  const int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;

  FILE *infile = fopen(binary_file, "rb");
  
  double correct_magic_number = 1.130814081305130e-9;
  double magic_number1, magic_number2, magic_number3;
  
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*RHOB_RGF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*P_RGF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*RHOB_LGF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*P_LGF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(&magic_number1, sizeof(double), 1, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*V_RU0GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*V_RU1GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*V_RU2GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*V_LU0GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*V_LU1GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*V_LU2GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(&magic_number2, sizeof(double), 1, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*B_RU0GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*B_RU1GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*B_RU2GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*B_LU0GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*B_LU1GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(auxevol_gfs + Nxx_plus_2NGHOSTS_tot*B_LU2GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
  fread(&magic_number3, sizeof(double), 1, infile);
  fclose(infile);
  if(magic_number1!=correct_magic_number){ printf("ERROR: magic_number1 does not match"); exit(1);}
  if(magic_number2!=correct_magic_number){ printf("ERROR: magic_number2 does not match"); exit(1);}
  if(magic_number3!=correct_magic_number){ printf("ERROR: magic_number3 does not match"); exit(1);}
"""

add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        name=name,
        params=params,
        body=body, 
        rel_path_to_Cparams=os.path.join("."))

In [10]:
    prefunc = r"""


static inline void read_from_binary_file_all   (const char *restrict binary_file, const paramstruct *params, double *restrict auxevol_gfs){
  #include "./set_Cparameters.h"


static inline void read_from_binary_file_recons(const char *restrict binary_file, const paramstruct *params, double *restrict auxevol_gfs){
  #include "./set_Cparameters.h"

}
"""

<a id='bc_functs'></a>

# Step 4: Set up boundary condition functions for chosen singular, curvilinear coordinate system \[Back to [top](#toc)\]
$$\label{bc_functs}$$

Next apply singular, curvilinear coordinate boundary conditions [as documented in the corresponding NRPy+ tutorial notebook](Tutorial-Start_to_Finish-Curvilinear_BCs.ipynb)

...But, for the moment, we're actually just using this because it writes the file `gridfunction_defines.h`.

<a id='mainc'></a>

# Step 5: `GiRaFFE_NRPy_standalone.c`: The Main C Code \[Back to [top](#toc)\]
$$\label{mainc}$$

In [11]:
def add_to_Cfunction_dict_main__IGM_Playground():
    includes = [  "NRPy_basic_defines.h", "NRPy_function_prototypes.h"]
    
    
    desc = """// main() function:
// Step 0: Read command-line input, set up grid structure, allocate memory for gridfunctions, set up coordinates
// Step 1: Write test data to gridfunctions
// Step 2: Overwrite all data in ghost zones with NaNs
// Step 3: Apply curvilinear boundary conditions
// Step 4: Print gridfunction data after curvilinear boundary conditions have been applied
// Step 5: Free all allocated memory
"""
    c_type = "int"
    name = "main"
    params = "int argc, const char *argv[]"
    prefunc = r"""
#define AM2 -0.0625
#define AM1  0.5625
#define A0   0.5625
#define A1  -0.0625
#define COMPUTE_FCVAL(METRICm2,METRICm1,METRIC,METRICp1) (AM2*(METRICm2) + AM1*(METRICm1) + A0*(METRIC) + A1*(METRICp1))

static inline void calculate_metric_face_values(const paramstruct *restrict params, 
                                  const int flux_dirn, const int metric_faces_gfs[],
                                  const int metric_gfs[],
                                  double *restrict auxevol_gfs) {
  #include "./set_Cparameters.h"

  LOOP_REGION(2, Nxx_plus_2NGHOSTS0 - 1,
              2, Nxx_plus_2NGHOSTS1 - 1,
              2, Nxx_plus_2NGHOSTS2 - 1) {

    int idxm2 = IDX3S(i0 - 2*kronecker_delta[flux_dirn+1][0], 
                      i1 - 2*kronecker_delta[flux_dirn+1][1], 
                      i2 - 2*kronecker_delta[flux_dirn+1][2]);
    
    int idxm1 = IDX3S(i0 - 1*kronecker_delta[flux_dirn+1][0], 
                      i1 - 1*kronecker_delta[flux_dirn+1][1], 
                      i2 - 1*kronecker_delta[flux_dirn+1][2]);

    int idx  = IDX3S(i0, i1, i2);

    int idxp1 = IDX3S(i0 + 1*kronecker_delta[flux_dirn+1][0], 
                      i1 + 1*kronecker_delta[flux_dirn+1][1], 
                      i2 + 1*kronecker_delta[flux_dirn+1][2]);

    for(int which_gf=0; which_gf<10; which_gf++){
      
      int gf_face = metric_faces_gfs[which_gf];
      int gf = metric_gfs[which_gf];
      auxevol_gfs[IDX4ptS(gf_face, idx)] = COMPUTE_FCVAL(auxevol_gfs[IDX4ptS(gf, idxm2)],
                                                         auxevol_gfs[IDX4ptS(gf, idxm1)],
                                                         auxevol_gfs[IDX4ptS(gf, idx  )],
                                                         auxevol_gfs[IDX4ptS(gf, idxp1)]);
    }
  }
}
"""
    body = r"""
  griddata_struct griddata;
  set_Cparameters_to_default(&griddata.params);

  // Step 0a: Read command-line input, error out if nonconformant
  /* if(argc != 5 || atoi(argv[1]) < NGHOSTS || atoi(argv[2]) < NGHOSTS || atoi(argv[3]) < NGHOSTS) {
    printf("Error: Expected one command-line argument: ./ScalarWaveCurvilinear_Playground Nx0 Nx1 Nx2,\n");
    printf("where Nx[0,1,2] is the number of grid points in the 0, 1, and 2 directions.\n");
    printf("Nx[] MUST BE larger than NGHOSTS (= %d)\n",NGHOSTS);
    exit(1);
  } */
  
  // Step 0b: Set up numerical grid structure, first in space...
  const int Nxx[3] = { atoi(argv[1]), atoi(argv[2]), atoi(argv[3]) };
  if(Nxx[0]%2 != 0 || Nxx[1]%2 != 0 || Nxx[2]%2 != 0) {
    printf("Error: Cannot guarantee a proper cell-centered grid if number of grid cells not set to even number.\n");
    printf("       For example, in case of angular directions, proper symmetry zones will not exist.\n");
    exit(1);
  }

  // Step 0c: Set free parameters, overwriting Cparameters defaults
  //          by hand or with command-line input, as desired.
#include "free_parameters.h"

  {
    int EigenCoord = 0;
    // Step 0d.ii: Call set_Nxx_dxx_invdx_params__and__xx(), which sets
    //             params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
    //             chosen Eigen-CoordSystem.
    set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &griddata.params, griddata.xx);
    // Step 0e: Find ghostzone mappings; set up bcstruct
    // Step 0e.i: Free allocated space for xx[][] array
  }
  
  // Step 0.m: Allocate memory for non y_n_gfs. We do this here to free up
  //         memory for setting up initial data (for cases in which initial
  //         data setup is memory intensive.)
  const int Nxx_plus_2NGHOSTS0 = griddata.params.Nxx_plus_2NGHOSTS0;
  const int Nxx_plus_2NGHOSTS1 = griddata.params.Nxx_plus_2NGHOSTS1;
  const int Nxx_plus_2NGHOSTS2 = griddata.params.Nxx_plus_2NGHOSTS2;
  const int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;

  double *restrict     evol_gfs    = (double *restrict)malloc(sizeof(double) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_tot);
  double *restrict etk_evol_gfs    = (double *restrict)malloc(sizeof(double) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_tot);
  double *restrict auxevol_gfs = (double *restrict)malloc(sizeof(double) * NUM_AUXEVOL_GFS * Nxx_plus_2NGHOSTS_tot);


  // Now we need to properly populate our structs for the unit test.

  reconstructed_prims_struct reconstructed_prims;
  reconstructed_prims_struct reconstructed_prims_r;
  reconstructed_prims_struct reconstructed_prims_l;

  conservative_fluxes_struct conservative_fluxes;
  conservative_sources_struct conservative_sources;

  metric_quantities_struct metric_quantities;
  metric_face_quantities_struct metric_face_quantities;
  metric_quantities_derivatives_struct metric_quantities_derivatives;

  // We begin by generating random inital data.

  for(int which_gf=0; which_gf<NUM_AUXEVOL_GFS; which_gf++) for(int idx=0; idx<Nxx_plus_2NGHOSTS_tot; idx++) {
    auxevol_gfs[IDX4ptS(which_gf, idx)] = 0.0 / 0.0; // ((double)rand() - (double)rand())/(double)(RAND_MAX);
        }

  for(int which_gf=0; which_gf<NUM_EVOL_GFS; which_gf++) for(int idx=0; idx<Nxx_plus_2NGHOSTS_tot; idx++) {
    evol_gfs[IDX4ptS(which_gf, idx)] = 0.0/0.0;
        }

  /*
  After reading in the same random data used in the original IGM code, now we
  calculate face cell-face values and derivatives at cell centers
  */

  int metric_faces_gfs[10] = {ALPHA_FACEGF, BETA_FACEU0GF, BETA_FACEU1GF, BETA_FACEU2GF,
                          GAMMA_FACEDD00GF, GAMMA_FACEDD01GF, GAMMA_FACEDD02GF,
                          GAMMA_FACEDD11GF, GAMMA_FACEDD12GF, GAMMA_FACEDD22GF};

  int metric_gfs[10] = {ALPHAGF, BETAU0GF, BETAU1GF, BETAU2GF,
                          GAMMADD00GF, GAMMADD01GF, GAMMADD02GF,
                          GAMMADD11GF, GAMMADD12GF, GAMMADD22GF};

  double dxxi[4] = {0, griddata.params.dxx0, griddata.params.dxx1, griddata.params.dxx2};

  int flux_dirn = 0;
  read_from_binary_file_all("input_data1.txt", &griddata.params, auxevol_gfs);

  calculate_metric_face_values(&griddata.params, 
                                flux_dirn, 
                                metric_faces_gfs,
                                metric_gfs,
                                auxevol_gfs);

  LOOP_REGION(NGHOSTS, Nxx_plus_2NGHOSTS0 - (NGHOSTS),
              NGHOSTS, Nxx_plus_2NGHOSTS1 - (NGHOSTS),
              NGHOSTS, Nxx_plus_2NGHOSTS2 - (NGHOSTS)) {
    
    int idx  = IDX3S(i0, i1, i2);

    auxevol_gfs[IDX4ptS(U4U0GF, idx)] = 0.47;
    auxevol_gfs[IDX4ptS(U4_RU0GF, idx)] = 0.87;
    auxevol_gfs[IDX4ptS(U4_LU0GF, idx)] = 0.49;

    auxevol_gfs[IDX4ptS(HGF, idx)] = 0.43;
    auxevol_gfs[IDX4ptS(H_RGF, idx)] = 0.67;
    auxevol_gfs[IDX4ptS(H_LGF, idx)] = 0.32;

    auxevol_gfs[IDX4ptS(GAMMA_TH_RGF, idx)] = 0.567;
    auxevol_gfs[IDX4ptS(GAMMA_TH_LGF, idx)] = 0.567;

    auxevol_gfs[IDX4ptS(EPSILON_TH_RGF, idx)] = 0.23;
    auxevol_gfs[IDX4ptS(EPSILON_TH_LGF, idx)] = 0.32;

    auxevol_gfs[IDX4ptS(DPCOLD_DRHOB_RGF, idx)] = 0.32;
    auxevol_gfs[IDX4ptS(DPCOLD_DRHOB_LGF, idx)] = 0.79;

    auxevol_gfs[IDX4ptS(U4U1GF, idx)] = auxevol_gfs[IDX4ptS(VU0GF, idx)]*auxevol_gfs[IDX4ptS(U4U0GF, idx)];
    auxevol_gfs[IDX4ptS(U4U2GF, idx)] = auxevol_gfs[IDX4ptS(VU1GF, idx)]*auxevol_gfs[IDX4ptS(U4U0GF, idx)];
    auxevol_gfs[IDX4ptS(U4U3GF, idx)] = auxevol_gfs[IDX4ptS(VU2GF, idx)]*auxevol_gfs[IDX4ptS(U4U0GF, idx)];
    auxevol_gfs[IDX4ptS(U4_RU1GF, idx)] = auxevol_gfs[IDX4ptS(V_RU0GF, idx)]*auxevol_gfs[IDX4ptS(U4_RU0GF, idx)];
    auxevol_gfs[IDX4ptS(U4_RU2GF, idx)] = auxevol_gfs[IDX4ptS(V_RU1GF, idx)]*auxevol_gfs[IDX4ptS(U4_RU0GF, idx)];
    auxevol_gfs[IDX4ptS(U4_RU3GF, idx)] = auxevol_gfs[IDX4ptS(V_RU2GF, idx)]*auxevol_gfs[IDX4ptS(U4_RU0GF, idx)];
    auxevol_gfs[IDX4ptS(U4_LU1GF, idx)] = auxevol_gfs[IDX4ptS(V_LU0GF, idx)]*auxevol_gfs[IDX4ptS(U4_LU0GF, idx)];
    auxevol_gfs[IDX4ptS(U4_LU2GF, idx)] = auxevol_gfs[IDX4ptS(V_LU1GF, idx)]*auxevol_gfs[IDX4ptS(U4_LU0GF, idx)];
    auxevol_gfs[IDX4ptS(U4_LU3GF, idx)] = auxevol_gfs[IDX4ptS(V_LU2GF, idx)]*auxevol_gfs[IDX4ptS(U4_LU0GF, idx)];

    initialize_structs(idx, &griddata.params, auxevol_gfs, 
                            &reconstructed_prims,
                            &reconstructed_prims_r,
                            &reconstructed_prims_l,
                            &metric_quantities,
                            &metric_face_quantities,
                            &metric_quantities_derivatives);

    calculate_HLLE_fluxes0(&reconstructed_prims_r, 
                           &reconstructed_prims_l,
                           &metric_face_quantities, 
                           &conservative_fluxes);

    auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED0GF, idx)]  = conservative_fluxes.HLLE_flux_StildeD0;
    auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED1GF, idx)]  = conservative_fluxes.HLLE_flux_StildeD1;
    auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED2GF, idx)]  = conservative_fluxes.HLLE_flux_StildeD2;
    auxevol_gfs[IDX4ptS(HLLE_FLUX_RHO_STARGF, idx)]  = conservative_fluxes.HLLE_flux_rho_star;
    auxevol_gfs[IDX4ptS(HLLE_FLUX_TAU_TILDEGF,idx)]  = conservative_fluxes.HLLE_flux_tau_tilde;
  }

  double inv_dxx = 1.0 / dxxi[flux_dirn+1];

  LOOP_REGION(NGHOSTS - 1, Nxx_plus_2NGHOSTS0 - 2,
              NGHOSTS - 1, Nxx_plus_2NGHOSTS1 - 2,
              NGHOSTS - 1, Nxx_plus_2NGHOSTS2 - 2) {
    
    int idxp1 = IDX3S(i0 + 1*kronecker_delta[flux_dirn+1][0], 
                      i1 + 1*kronecker_delta[flux_dirn+1][1], 
                      i2 + 1*kronecker_delta[flux_dirn+1][2]);

    int idx  = IDX3S(i0, i1, i2);

    evol_gfs[IDX4ptS(STILDED0GF, idx)]  = inv_dxx*(auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED0GF, idx)] - auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED0GF, idxp1)]);
    evol_gfs[IDX4ptS(STILDED1GF, idx)]  = inv_dxx*(auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED1GF, idx)] - auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED1GF, idxp1)]);
    evol_gfs[IDX4ptS(STILDED2GF, idx)]  = inv_dxx*(auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED2GF, idx)] - auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED2GF, idxp1)]);
    evol_gfs[IDX4ptS(RHO_STARGF, idx)]  = inv_dxx*(auxevol_gfs[IDX4ptS(HLLE_FLUX_RHO_STARGF, idx)] - auxevol_gfs[IDX4ptS(HLLE_FLUX_RHO_STARGF, idxp1)]);
    evol_gfs[IDX4ptS(TAU_TILDEGF,idx)]  = inv_dxx*(auxevol_gfs[IDX4ptS(HLLE_FLUX_TAU_TILDEGF,idx)] - auxevol_gfs[IDX4ptS(HLLE_FLUX_TAU_TILDEGF,idxp1)]);


    auxevol_gfs[IDX4ptS(ALPHA_DD0GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(ALPHA_FACEGF, idxp1)]
                                                    - auxevol_gfs[IDX4ptS(ALPHA_FACEGF, idx  )]);

    auxevol_gfs[IDX4ptS(BETAU_DD00GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(BETA_FACEU0GF, idxp1)]
                                                     - auxevol_gfs[IDX4ptS(BETA_FACEU0GF, idx  )]);

    auxevol_gfs[IDX4ptS(BETAU_DD10GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(BETA_FACEU1GF, idxp1)]
                                                     - auxevol_gfs[IDX4ptS(BETA_FACEU1GF, idx  )]);

    auxevol_gfs[IDX4ptS(BETAU_DD20GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(BETA_FACEU2GF, idxp1)]
                                                     - auxevol_gfs[IDX4ptS(BETA_FACEU2GF, idx  )]);


    auxevol_gfs[IDX4ptS(GAMMADD_DD000GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD00GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD00GF, idx  )]);

    auxevol_gfs[IDX4ptS(GAMMADD_DD010GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD01GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD01GF, idx  )]);

    auxevol_gfs[IDX4ptS(GAMMADD_DD020GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD02GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD02GF, idx  )]);

    auxevol_gfs[IDX4ptS(GAMMADD_DD110GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD11GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD11GF, idx  )]);

    auxevol_gfs[IDX4ptS(GAMMADD_DD120GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD12GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD12GF, idx  )]);

    auxevol_gfs[IDX4ptS(GAMMADD_DD220GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD22GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD22GF, idx  )]);
  }


  flux_dirn = 1;
  read_from_binary_file_recons("input_data2.txt", &griddata.params, auxevol_gfs);

  calculate_metric_face_values(&griddata.params, 
                                flux_dirn, 
                                metric_faces_gfs,
                                metric_gfs,
                                auxevol_gfs);

  
  LOOP_REGION(NGHOSTS - 1, Nxx_plus_2NGHOSTS0 - 2,
              NGHOSTS - 1, Nxx_plus_2NGHOSTS1 - 2,
              NGHOSTS - 1, Nxx_plus_2NGHOSTS2 - 2) {
    
    int idx  = IDX3S(i0, i1, i2);

    auxevol_gfs[IDX4ptS(U4_RU1GF, idx)] = auxevol_gfs[IDX4ptS(V_RU0GF, idx)]*auxevol_gfs[IDX4ptS(U4_RU0GF, idx)];
    auxevol_gfs[IDX4ptS(U4_RU2GF, idx)] = auxevol_gfs[IDX4ptS(V_RU1GF, idx)]*auxevol_gfs[IDX4ptS(U4_RU0GF, idx)];
    auxevol_gfs[IDX4ptS(U4_RU3GF, idx)] = auxevol_gfs[IDX4ptS(V_RU2GF, idx)]*auxevol_gfs[IDX4ptS(U4_RU0GF, idx)];
    auxevol_gfs[IDX4ptS(U4_LU1GF, idx)] = auxevol_gfs[IDX4ptS(V_LU0GF, idx)]*auxevol_gfs[IDX4ptS(U4_LU0GF, idx)];
    auxevol_gfs[IDX4ptS(U4_LU2GF, idx)] = auxevol_gfs[IDX4ptS(V_LU1GF, idx)]*auxevol_gfs[IDX4ptS(U4_LU0GF, idx)];
    auxevol_gfs[IDX4ptS(U4_LU3GF, idx)] = auxevol_gfs[IDX4ptS(V_LU2GF, idx)]*auxevol_gfs[IDX4ptS(U4_LU0GF, idx)];

    initialize_structs(idx, &griddata.params, auxevol_gfs, 
                            &reconstructed_prims,
                            &reconstructed_prims_r,
                            &reconstructed_prims_l,
                            &metric_quantities,
                            &metric_face_quantities,
                            &metric_quantities_derivatives);

    calculate_HLLE_fluxes1(&reconstructed_prims_r, 
                           &reconstructed_prims_l,
                           &metric_face_quantities, 
                           &conservative_fluxes);

    auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED0GF, idx)]  = conservative_fluxes.HLLE_flux_StildeD0;
    auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED1GF, idx)]  = conservative_fluxes.HLLE_flux_StildeD1;
    auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED2GF, idx)]  = conservative_fluxes.HLLE_flux_StildeD2;
    auxevol_gfs[IDX4ptS(HLLE_FLUX_RHO_STARGF, idx)]  = conservative_fluxes.HLLE_flux_rho_star;
    auxevol_gfs[IDX4ptS(HLLE_FLUX_TAU_TILDEGF,idx)]  = conservative_fluxes.HLLE_flux_tau_tilde;
  }

  inv_dxx = 1.0 / dxxi[flux_dirn+1];

  LOOP_REGION(NGHOSTS - 1, Nxx_plus_2NGHOSTS0 - 2,
              NGHOSTS - 1, Nxx_plus_2NGHOSTS1 - 2,
              NGHOSTS - 1, Nxx_plus_2NGHOSTS2 - 2) {
    
    int idxp1 = IDX3S(i0 + 1*kronecker_delta[flux_dirn+1][0], 
                      i1 + 1*kronecker_delta[flux_dirn+1][1], 
                      i2 + 1*kronecker_delta[flux_dirn+1][2]);

    int idx  = IDX3S(i0, i1, i2);

    evol_gfs[IDX4ptS(STILDED0GF, idx)]  += inv_dxx*(auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED0GF, idx)] - auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED0GF, idxp1)]);
    evol_gfs[IDX4ptS(STILDED1GF, idx)]  += inv_dxx*(auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED1GF, idx)] - auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED1GF, idxp1)]);
    evol_gfs[IDX4ptS(STILDED2GF, idx)]  += inv_dxx*(auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED2GF, idx)] - auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED2GF, idxp1)]);
    evol_gfs[IDX4ptS(RHO_STARGF, idx)]  += inv_dxx*(auxevol_gfs[IDX4ptS(HLLE_FLUX_RHO_STARGF, idx)] - auxevol_gfs[IDX4ptS(HLLE_FLUX_RHO_STARGF, idxp1)]);
    evol_gfs[IDX4ptS(TAU_TILDEGF,idx)]  += inv_dxx*(auxevol_gfs[IDX4ptS(HLLE_FLUX_TAU_TILDEGF,idx)] - auxevol_gfs[IDX4ptS(HLLE_FLUX_TAU_TILDEGF,idxp1)]);


    auxevol_gfs[IDX4ptS(ALPHA_DD1GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(ALPHA_FACEGF, idxp1)]
                                                    - auxevol_gfs[IDX4ptS(ALPHA_FACEGF, idx  )]);

    auxevol_gfs[IDX4ptS(BETAU_DD01GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(BETA_FACEU0GF, idxp1)]
                                                     - auxevol_gfs[IDX4ptS(BETA_FACEU0GF, idx  )]);

    auxevol_gfs[IDX4ptS(BETAU_DD11GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(BETA_FACEU1GF, idxp1)]
                                                     - auxevol_gfs[IDX4ptS(BETA_FACEU1GF, idx  )]);

    auxevol_gfs[IDX4ptS(BETAU_DD21GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(BETA_FACEU2GF, idxp1)]
                                                     - auxevol_gfs[IDX4ptS(BETA_FACEU2GF, idx  )]);


    auxevol_gfs[IDX4ptS(GAMMADD_DD001GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD00GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD00GF, idx  )]);

    auxevol_gfs[IDX4ptS(GAMMADD_DD011GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD01GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD01GF, idx  )]);

    auxevol_gfs[IDX4ptS(GAMMADD_DD021GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD02GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD02GF, idx  )]);

    auxevol_gfs[IDX4ptS(GAMMADD_DD111GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD11GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD11GF, idx  )]);

    auxevol_gfs[IDX4ptS(GAMMADD_DD121GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD12GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD12GF, idx  )]);

    auxevol_gfs[IDX4ptS(GAMMADD_DD221GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD22GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD22GF, idx  )]);
  }

  flux_dirn = 2;
  read_from_binary_file_recons("input_data3.txt", &griddata.params, auxevol_gfs);

  calculate_metric_face_values(&griddata.params, 
                                flux_dirn, 
                                metric_faces_gfs,
                                metric_gfs,
                                auxevol_gfs);

  LOOP_REGION(NGHOSTS - 1, Nxx_plus_2NGHOSTS0 - 2,
              NGHOSTS - 1, Nxx_plus_2NGHOSTS1 - 2,
              NGHOSTS - 1, Nxx_plus_2NGHOSTS2 - 2) {

    int idx  = IDX3S(i0, i1, i2);

    auxevol_gfs[IDX4ptS(U4_RU1GF, idx)] = auxevol_gfs[IDX4ptS(V_RU0GF, idx)]*auxevol_gfs[IDX4ptS(U4_RU0GF, idx)];
    auxevol_gfs[IDX4ptS(U4_RU2GF, idx)] = auxevol_gfs[IDX4ptS(V_RU1GF, idx)]*auxevol_gfs[IDX4ptS(U4_RU0GF, idx)];
    auxevol_gfs[IDX4ptS(U4_RU3GF, idx)] = auxevol_gfs[IDX4ptS(V_RU2GF, idx)]*auxevol_gfs[IDX4ptS(U4_RU0GF, idx)];
    auxevol_gfs[IDX4ptS(U4_LU1GF, idx)] = auxevol_gfs[IDX4ptS(V_LU0GF, idx)]*auxevol_gfs[IDX4ptS(U4_LU0GF, idx)];
    auxevol_gfs[IDX4ptS(U4_LU2GF, idx)] = auxevol_gfs[IDX4ptS(V_LU1GF, idx)]*auxevol_gfs[IDX4ptS(U4_LU0GF, idx)];
    auxevol_gfs[IDX4ptS(U4_LU3GF, idx)] = auxevol_gfs[IDX4ptS(V_LU2GF, idx)]*auxevol_gfs[IDX4ptS(U4_LU0GF, idx)];

    initialize_structs(idx, &griddata.params, auxevol_gfs, 
                            &reconstructed_prims,
                            &reconstructed_prims_r,
                            &reconstructed_prims_l,
                            &metric_quantities,
                            &metric_face_quantities,
                            &metric_quantities_derivatives);

    calculate_HLLE_fluxes2(&reconstructed_prims_r, 
                           &reconstructed_prims_l,
                           &metric_face_quantities, 
                           &conservative_fluxes);

    auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED0GF, idx)]  = conservative_fluxes.HLLE_flux_StildeD0;
    auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED1GF, idx)]  = conservative_fluxes.HLLE_flux_StildeD1;
    auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED2GF, idx)]  = conservative_fluxes.HLLE_flux_StildeD2;
    auxevol_gfs[IDX4ptS(HLLE_FLUX_RHO_STARGF, idx)]  = conservative_fluxes.HLLE_flux_rho_star;
    auxevol_gfs[IDX4ptS(HLLE_FLUX_TAU_TILDEGF,idx)]  = conservative_fluxes.HLLE_flux_tau_tilde;

    }

  inv_dxx = 1.0 / dxxi[flux_dirn+1];

  LOOP_REGION(NGHOSTS - 1, Nxx_plus_2NGHOSTS0 - 2,
              NGHOSTS - 1, Nxx_plus_2NGHOSTS1 - 2,
              NGHOSTS - 1, Nxx_plus_2NGHOSTS2 - 2) {
    
    int idxp1 = IDX3S(i0 + 1*kronecker_delta[flux_dirn+1][0], 
                      i1 + 1*kronecker_delta[flux_dirn+1][1], 
                      i2 + 1*kronecker_delta[flux_dirn+1][2]);

    int idx  = IDX3S(i0, i1, i2);

    evol_gfs[IDX4ptS(STILDED0GF, idx)]  += inv_dxx*(auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED0GF, idx)] - auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED0GF, idxp1)]);
    evol_gfs[IDX4ptS(STILDED1GF, idx)]  += inv_dxx*(auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED1GF, idx)] - auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED1GF, idxp1)]);
    evol_gfs[IDX4ptS(STILDED2GF, idx)]  += inv_dxx*(auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED2GF, idx)] - auxevol_gfs[IDX4ptS(HLLE_FLUX_STILDED2GF, idxp1)]);
    evol_gfs[IDX4ptS(RHO_STARGF, idx)]  += inv_dxx*(auxevol_gfs[IDX4ptS(HLLE_FLUX_RHO_STARGF, idx)] - auxevol_gfs[IDX4ptS(HLLE_FLUX_RHO_STARGF, idxp1)]);
    evol_gfs[IDX4ptS(TAU_TILDEGF,idx)]  += inv_dxx*(auxevol_gfs[IDX4ptS(HLLE_FLUX_TAU_TILDEGF,idx)] - auxevol_gfs[IDX4ptS(HLLE_FLUX_TAU_TILDEGF,idxp1)]);


    auxevol_gfs[IDX4ptS(ALPHA_DD2GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(ALPHA_FACEGF, idxp1)]
                                                    - auxevol_gfs[IDX4ptS(ALPHA_FACEGF, idx  )]);

    auxevol_gfs[IDX4ptS(BETAU_DD02GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(BETA_FACEU0GF, idxp1)]
                                                     - auxevol_gfs[IDX4ptS(BETA_FACEU0GF, idx  )]);

    auxevol_gfs[IDX4ptS(BETAU_DD12GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(BETA_FACEU1GF, idxp1)]
                                                     - auxevol_gfs[IDX4ptS(BETA_FACEU1GF, idx  )]);

    auxevol_gfs[IDX4ptS(BETAU_DD22GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(BETA_FACEU2GF, idxp1)]
                                                     - auxevol_gfs[IDX4ptS(BETA_FACEU2GF, idx  )]);


    auxevol_gfs[IDX4ptS(GAMMADD_DD002GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD00GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD00GF, idx  )]);

    auxevol_gfs[IDX4ptS(GAMMADD_DD012GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD01GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD01GF, idx  )]);

    auxevol_gfs[IDX4ptS(GAMMADD_DD022GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD02GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD02GF, idx  )]);

    auxevol_gfs[IDX4ptS(GAMMADD_DD112GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD11GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD11GF, idx  )]);

    auxevol_gfs[IDX4ptS(GAMMADD_DD122GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD12GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD12GF, idx  )]);

    auxevol_gfs[IDX4ptS(GAMMADD_DD222GF, idx)] = inv_dxx*(auxevol_gfs[IDX4ptS(GAMMA_FACEDD22GF, idxp1)]
                                                        - auxevol_gfs[IDX4ptS(GAMMA_FACEDD22GF, idx  )]);
  }

    LOOP_REGION(NGHOSTS - 1, Nxx_plus_2NGHOSTS0 - 2,
                NGHOSTS - 1, Nxx_plus_2NGHOSTS1 - 2,
                NGHOSTS - 1, Nxx_plus_2NGHOSTS2 - 2) {
        int idx  = IDX3S(i0, i1, i2);
    
      initialize_structs(idx, &griddata.params, auxevol_gfs, &reconstructed_prims,
                           &reconstructed_prims_r,
                           &reconstructed_prims_l,
                           &metric_quantities,
                           &metric_face_quantities,
                           &metric_quantities_derivatives);
                           
           
      calculate_all_source_terms(&reconstructed_prims,
                                 &metric_quantities,
                                 &metric_quantities_derivatives,
                                 &conservative_sources);
                                  
      evol_gfs[IDX4ptS(STILDED0GF, idx)]  += conservative_sources.StildeD0_src;
      evol_gfs[IDX4ptS(STILDED1GF, idx)]  += conservative_sources.StildeD1_src;
      evol_gfs[IDX4ptS(STILDED2GF, idx)]  += conservative_sources.StildeD2_src;
      evol_gfs[IDX4ptS(TAU_TILDEGF, idx)] += conservative_sources.tau_tilde_src;
    }

    FILE *infile = fopen("output_rhs_data.txt", "rb");
    double rhs_correct_magic_number = 9.524300707856655e-3;
    double rhs_magic_number1, rhs_magic_number2, rhs_magic_number3;
    fread(&rhs_magic_number1, sizeof(double), 1, infile);
    fread(etk_evol_gfs + Nxx_plus_2NGHOSTS_tot*RHO_STARGF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
    fread(etk_evol_gfs + Nxx_plus_2NGHOSTS_tot*TAU_TILDEGF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
    fread(etk_evol_gfs + Nxx_plus_2NGHOSTS_tot*STILDED0GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
    fread(&rhs_magic_number2, sizeof(double), 1, infile);
    fread(etk_evol_gfs + Nxx_plus_2NGHOSTS_tot*STILDED1GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
    fread(etk_evol_gfs + Nxx_plus_2NGHOSTS_tot*STILDED2GF, sizeof(double), Nxx_plus_2NGHOSTS_tot, infile);
    fread(&rhs_magic_number3, sizeof(double), 1, infile);
    fclose(infile);
    if(rhs_magic_number1!=rhs_correct_magic_number){ printf("ERROR: magic_number1 does not match"); exit(1);}
    if(rhs_magic_number2!=rhs_correct_magic_number){ printf("ERROR: magic_number2 does not match"); exit(1);}
    if(rhs_magic_number3!=rhs_correct_magic_number){ printf("ERROR: magic_number3 does not match"); exit(1);}

    LOOP_REGION(NGHOSTS, Nxx_plus_2NGHOSTS0 - NGHOSTS-1,
                NGHOSTS, Nxx_plus_2NGHOSTS1 - NGHOSTS-1,
                NGHOSTS, Nxx_plus_2NGHOSTS2 - NGHOSTS-1) {

      int idx  = IDX3S(i0, i1, i2);
      printf("%d %d %d %.3f %.3f %.3f %.3f %.3f\n", 
      i0, i1, i2,
      log10(fabs(0.5*(evol_gfs[IDX4ptS(STILDED0GF, idx)] - etk_evol_gfs[IDX4ptS(STILDED0GF, idx)]) / (evol_gfs[IDX4ptS(STILDED0GF, idx)] + etk_evol_gfs[IDX4ptS(STILDED0GF, idx)]))),
      log10(fabs(0.5*(evol_gfs[IDX4ptS(STILDED1GF, idx)] - etk_evol_gfs[IDX4ptS(STILDED1GF, idx)]) / (evol_gfs[IDX4ptS(STILDED1GF, idx)] + etk_evol_gfs[IDX4ptS(STILDED1GF, idx)]))),
      log10(fabs(0.5*(evol_gfs[IDX4ptS(STILDED2GF, idx)] - etk_evol_gfs[IDX4ptS(STILDED2GF, idx)]) / (evol_gfs[IDX4ptS(STILDED2GF, idx)] + etk_evol_gfs[IDX4ptS(STILDED2GF, idx)]))),
      log10(fabs(0.5*(evol_gfs[IDX4ptS(TAU_TILDEGF, idx)] - etk_evol_gfs[IDX4ptS(TAU_TILDEGF, idx)]) / (evol_gfs[IDX4ptS(TAU_TILDEGF, idx)] + etk_evol_gfs[IDX4ptS(TAU_TILDEGF, idx)]))),
      log10(fabs(0.5*(evol_gfs[IDX4ptS(RHO_STARGF, idx)] - etk_evol_gfs[IDX4ptS(RHO_STARGF, idx)]) / (evol_gfs[IDX4ptS(RHO_STARGF, idx)] + etk_evol_gfs[IDX4ptS(RHO_STARGF, idx)]))));
  }

 
  // Step 4: Free all allocated memory
  free(evol_gfs);
  free(auxevol_gfs);
  for(int i=0;i<3;i++) free(griddata.xx[i]);
  return 0;
"""
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        prefunc=prefunc,
        body=body,
        rel_path_to_Cparams=os.path.join("."), enableCparameters=False)

In [12]:
# Generate & register the find_timestep() function
rfm.register_C_functions(enable_rfm_precompute=enable_rfm_precompute,
                         use_unit_wavespeed_for_find_timestep=True)
rfm.register_NRPy_basic_defines(enable_rfm_precompute=enable_rfm_precompute)


In [13]:
import outputC as outC
outC.outputC_register_C_functions_and_NRPy_basic_defines()  # #define M_PI,  etc.
# Declare paramstruct, register set_Cparameters_to_default(),
#   and output declare_Cparameters_struct.h and set_Cparameters[].h:
outC.NRPy_param_funcs_register_C_functions_and_NRPy_basic_defines(os.path.join(Ccodesrootdir))

gri.register_C_functions_and_NRPy_basic_defines(enable_griddata_struct=True,
                                                enable_bcstruct_in_griddata_struct=False,
                                                enable_rfmstruct=enable_rfm_precompute,
                                                enable_MoL_gfs_struct=False)  # #define IDX3S(),  etc.
fin.register_C_functions_and_NRPy_basic_defines(NGHOSTS_account_for_onezone_upwind=False)  # #define NGHOSTS, etc.



# Call this last: Set up NRPy_basic_defines.h and NRPy_function_prototypes.h.
outC.construct_NRPy_basic_defines_h(Ccodesrootdir, enable_SIMD=enable_SIMD)
outC.construct_NRPy_function_prototypes_h(Ccodesrootdir)

add_to_Cfunction_dict_main__IGM_Playground()


In [14]:
metric_derivs_str = ""
for var in all_free_sysmbols:
    if "_dD" in str(var):
        metric_derivs_str += "double "+str(var)+";\n"
        

module_structs_str = r"""

//#ifndef TINYDOUBLE
static const double TINYDOUBLE = 1e-100;
//#endif

//#ifndef sqrt4pi
static const double sqrt4pi = 3.544907701811032054596334966682290365L;
//#endif

static const int kronecker_delta[4][3] = { { 0,0,0 },
                                    { 1,0,0 },
                                    { 0,1,0 },
                                    { 0,0,1 } };

/*
   The struct reconstructed_prims_struct contains variables for storing the (point-wise)
   reconstructed primitive variables. The struct elements are detailed below:
   
 --rhob: the baryonic density
 
 --P: the pressure
 
 --u4U*: the contravariant fluid 4-velocity
   
 --BU*: the contravariant magnetic field
 
 --h: the enthalpy
 
 --Gamma_th: an EOS quantity
 
 --epsilon_th: an EOS quantity
 
 --dPcold_drhob: an EOS quantity
*/

typedef struct __reconstructed_prims__ {
double u4U0, u4U1, u4U2, u4U3;
double BU0, BU1, BU2;
double rhob;
double P, h, Gamma_th, epsilon_th, dPcold_drhob;
} reconstructed_prims_struct;


/*
   Parameters used throughout the module. The struct elements are detailed below:
 
 --TINYDOUBLE: tiny double to avoid divide by zero issues
 
 --sqrt4pi: \sqrt{4\pi} to 16 S.D.
*/

/*
typedef struct __rhss_params__ {
double TINYDOUBLE, sqrt4pi;
} rhss_params_struct;
*/


/*
   The struct conservative_fluxes_struct contains variables for HLLE fluxes that the user
   will finite difference later. Note that we do not store the fluxes for all three
   directions. The struct elements are detailed below:
   
 --cmin* / cmax*: the minimum and maximum characteristic speeds for all three flux directions
 
 --HLLE_flux_StildeD*: the HLLE flux for the momentum term term, all three components
 
 --HLLE_flux_rho_star: the HLLE flux for the baryonic density
   
 --HLLE_flux_tau_tilde: the HLLE flux for the energy
*/

typedef struct __conservative_fluxes__ {
double cmin_dirn0, cmin_dirn1, cmin_dirn2;
double cmax_dirn0, cmax_dirn1, cmax_dirn2;
double HLLE_flux_StildeD0, HLLE_flux_StildeD1, HLLE_flux_StildeD2;
double HLLE_flux_rho_star, HLLE_flux_tau_tilde;
} conservative_fluxes_struct;


/*
   The struct conservative_sources_struct contains stores the point-wise values 
   of the source terms for the momentum and energy equations. The struct 
   elements are detailed below:
   
 --StildeD*_src: momentum equations source terms
 
 --tau_tilde_src: tau equation source term
*/

typedef struct __conservative_sources__ {
double StildeD0_src, StildeD1_src, StildeD2_src;
double tau_tilde_src;
} conservative_sources_struct;


"""
if formalism=="BSSN":
    module_structs_str += r"""
/*
   The struct metric_quantities_struct contains variables for storing the (point-wise)
   metric variables. Note that while we use the names of rescaled BSSN variables, 
   since this module is to be used in Cartesian coordinates only, they are not actually
   rescaled. The struct elements are detailed below:
   
 --alpha: the lapse
 
 --trK: the trace of the extrinsic curvature
 
 --cf: the conformal factor, assuming cf = e^4\phi
 
 --vetU*: the rescaled shift vector
   
 --aDD**: the rescaled trace-free part of the extrinsic curvature
 
 --hDD*: the rescaled deviations from the background metric
*/

typedef struct __metric_quantities__ {
double alpha, trK, cf;
double vetU0, vetU1, vetU2;
double aDD01, aDD02, aDD12, aDD00, aDD11, aDD22;
double hDD01, hDD02, hDD12, hDD00, hDD11, hDD22;
} metric_quantities_struct;


/*
   The struct metric_face_quantities_struct contains metric quantities stored on grid faces.
*/

typedef struct __metric_face_quantities__ {
double alpha_face, cf_face;
double vet_faceU0, vet_faceU1, vet_faceU2;
double h_faceDD01, h_faceDD02, h_faceDD12, h_faceDD00, h_faceDD11, h_faceDD22;
} metric_face_quantities_struct;


"""
elif formalism=="ADM":
    module_structs_str += r"""
/*
   The struct metric_quantities_struct contains variables for storing the (point-wise)
   metric variables. The struct elements are detailed below:
   
 --alpha: the lapse
 
 --betaU*: the shift vector
   
 --gammaDD**: the physical 3-metric
 
 --KDD*: the extrinsic curvature
*/

typedef struct __metric_quantities__ {
double alpha;
double betaU0, betaU1, betaU2;
double gammaDD01, gammaDD02, gammaDD12, gammaDD00, gammaDD11, gammaDD22;
double KDD01, KDD02, KDD12, KDD00, KDD11, KDD22;
} metric_quantities_struct;


/*
   The struct metric_face_quantities_struct contains metric quantities stored on grid faces.
*/

typedef struct __metric_face_quantities__ {
double alpha_face;
double beta_faceU0, beta_faceU1, beta_faceU2;
double gamma_faceDD01, gamma_faceDD02, gamma_faceDD12, gamma_faceDD00, gamma_faceDD11, gamma_faceDD22;
} metric_face_quantities_struct;

"""
else:
    print("ERROR: invalid choice of formalism")
    sys.exit(1)

module_structs_str += r"""
/*
   The struct metric_quantities_derivatives_struct contains variables for
   precomputed derivatives of the metric quantities.
*/

typedef struct __metric_quantities_derivatives__ {
"""+metric_derivs_str+"""
} metric_quantities_derivatives_struct;

"""

with open(os.path.join(Ccodesrootdir,"NRPy_basic_defines.h"), "a") as file:
    file.write(module_structs_str)

<a id='compileexec'></a>

# Step 6: Compile generated C codes & perform GRFFE simulations \[Back to [top](#toc)\]
$$\label{compileexec}$$

To aid in the cross-platform-compatible (with Windows, MacOS, & Linux) compilation and execution, we make use of `cmdline_helper` [(**Tutorial**)](Tutorial-cmdline_helper.ipynb).

In [15]:
import cmdline_helper as cmd
cmd.new_C_compile(Ccodesrootdir, "IGM_standalone",
                  uses_free_parameters_h=True, compiler_opt_option="debug") # fastdebug or debug also supported
os.chdir(Ccodesrootdir)
# Clean up existing output files
cmd.delete_existing_files("out*.txt")
cmd.delete_existing_files("out*.png")

(EXEC): Executing `make -j34`...
(BENCH): Finished executing in 0.8 seconds.
Finished compilation.


In [16]:
cmd.Execute("IGM_standalone", "8 8 8 ", os.path.join("output", "out8.txt"))

# Return to root directory
os.chdir(os.path.join(".."))

(EXEC): Executing `taskset -c 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15 ./IGM_standalone 8 8 8 `...
(BENCH): Finished executing in 0.2 seconds.


<a id='latex_pdf_output'></a>

# Step 8: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-Start_to_Finish-GiRaFFE_NRPy-1D_tests-staggered.pdf](Tutorial-Start_to_Finish-GiRaFFE_NRPy-1D_tests-staggered.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [17]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-Start_to_Finish-GiRaFFE_NRPy-1D_tests-staggered",location_of_template_file=os.path.join(".."))

TypeError: output_Jupyter_notebook_to_LaTeXed_PDF() got an unexpected keyword argument 'location_of_template_file'